# Simulation Example

This notebook is a simple example of how bioRSP's radar scanning mechanism works with a simulated dataset of background and foreground points. Points are generated in a unit circle and optionally clustered around centroids. The scanning window is then animated, and histograms and CDFs are computed for each frame.

In [ ]:
from typing import List, Tuple, Optional

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import biorsp as rsp

In [ ]:
def main() -> None:
    """
    Main function to generate background and foreground points, convert them to polar coordinates,
    animate the scanning window, and plot histograms and CDFs for each frame.
    """
    MODE = rsp.get_param("MODE")
    SCANNING_WINDOW = rsp.get_param("SCANNING_WINDOW")
    RESOLUTION = rsp.get_param("RESOLUTION")

    bg_size: int = 10000
    coverage: float = 0.1  # 10% coverage by foreground
    distribution: str = "clustered"  # 'uniform' or 'clustered'
    centroids: List[Tuple[float, float]] = [(0.5, 0.5), (0, -1)]
    seed: Optional[int] = 42

    # Generate synthetic background and foreground points
    background_points, foreground_points = rsp.generate_points(
        bg_size=bg_size,
        coverage=coverage,
        distribution=distribution,
        centroids=centroids,
        seed=seed,
    )

    # Vantage point: center of the background distribution
    vantage_point: Tuple[float, float] = tuple(np.mean(background_points, axis=0))

    # Convert Cartesian to polar coordinates relative to vantage point
    r_fg, theta_fg = rsp.cartesian_to_polar(foreground_points, vantage_point)
    r_bg, theta_bg = rsp.cartesian_to_polar(background_points, vantage_point)

    # Define bin edges in radians. For plotting histograms, we will convert to degrees later.
    BIN_SIZE = SCANNING_WINDOW / RESOLUTION
    BIN_EDGES = np.arange(
        -SCANNING_WINDOW / 2, SCANNING_WINDOW / 2 + BIN_SIZE, BIN_SIZE
    )
    BIN_EDGES_DEG = np.degrees(BIN_EDGES)  # For histograms and CDF visualization

    # Angles at which we will rotate the scanning window
    theta_k_list = np.linspace(0, 2 * np.pi, RESOLUTION, endpoint=False)

    # Compute the maximum radius for plotting limits
    radius_max: float = max(np.max(r_fg), np.max(r_bg))
    rsp_diffs = np.zeros(RESOLUTION)

    fig: plt.Figure = plt.figure(figsize=(24, 8))

    # Panel 1: Polar plot showing the scanning window and points
    ax1: plt.Axes = plt.subplot(1, 4, 1, projection="polar")
    ax1.set_title("Scanning Window", va="bottom")

    # Panel 2: Histogram of angles within the scanning window
    ax2: plt.Axes = plt.subplot(1, 4, 2)
    ax2.set_title("Histogram of Angles within Scanning Window")
    ax2.set_xlabel("Angle within Scanning Window (Degrees)")
    ax2.set_ylabel("Count")
    ax2.grid(True, linestyle="--", alpha=0.5)

    # Panel 3: CDF of angles within the scanning window
    ax3: plt.Axes = plt.subplot(1, 4, 3)
    ax3.set_title("CDF of Angles within Scanning Window")
    ax3.set_xlabel("Normalized Angle (0 to 1)")
    ax3.set_ylabel("CDF")
    ax3.set_ylim(0, 1)
    ax3.grid(True, linestyle="--", alpha=0.5)

    # Panel 4: Polar plot of RSP Diff over angles
    ax4: plt.Axes = plt.subplot(1, 4, 4, polar=True)
    ax4.set_title("In-progress RSP Diff Plot", va="bottom")

    def plot_scanning_window(
        ax: plt.Axes, start_angle: float, end_angle: float, radius_max: float
    ) -> None:
        """
        Fill the scanning window region on a polar plot.
        """
        if start_angle < end_angle:
            theta_window: np.ndarray = np.linspace(start_angle, end_angle, 100)
        else:
            # Handle wrap-around by adding 2*pi to end_angle
            theta_window: np.ndarray = np.linspace(
                start_angle, end_angle + 2 * np.pi, 100
            ) % (2 * np.pi)
        r_window: np.ndarray = np.full_like(theta_window, radius_max)
        theta_polygon: np.ndarray = np.concatenate(
            ([start_angle], theta_window, [end_angle], [start_angle])
        )
        r_polygon: np.ndarray = np.concatenate(([0], r_window, [0], [0]))
        ax.fill(
            theta_polygon, r_polygon, color="yellow", alpha=0.1, label="Scanning Window"
        )

    def animate(i: int) -> None:
        angle: float = theta_k_list[i]
        start_angle: float = (angle - SCANNING_WINDOW / 2) % (2 * np.pi)
        end_angle: float = (angle + SCANNING_WINDOW / 2) % (2 * np.pi)

        ax1.clear()
        ax1.set_title("Scanning Window", va="bottom")
        ax1.set_ylim(0, radius_max * 1.1)
        ax1.grid(True)

        ax1.scatter(theta_bg, r_bg, color="grey", s=1, label="Background", alpha=0.25)
        ax1.scatter(theta_fg, r_fg, color="red", s=1, label="Foreground", alpha=0.5)

        plot_scanning_window(ax1, start_angle, end_angle, radius_max)

        fg_in_window = rsp.within_angle(theta_fg, angle, SCANNING_WINDOW)
        bg_in_window = rsp.within_angle(theta_bg, angle, SCANNING_WINDOW)

        ax1.scatter(
            theta_bg[bg_in_window],
            r_bg[bg_in_window],
            color="grey",
            s=1,
            alpha=0.75,
            label="BG in Window",
        )
        ax1.scatter(
            theta_fg[fg_in_window],
            r_fg[fg_in_window],
            color="red",
            s=1,
            alpha=1.0,
            label="FG in Window",
        )

        handles, labels = ax1.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        ax1.legend(
            by_label.values(),
            by_label.keys(),
            loc="upper right",
            bbox_to_anchor=(1.1, 1.1),
        )

        fg_angles_in_window = theta_fg[fg_in_window]
        bg_angles_in_window = theta_bg[bg_in_window]

        # Compute relative angles in radians
        relative_theta_fg = (fg_angles_in_window - angle + np.pi) % (2 * np.pi) - np.pi
        relative_theta_bg = (bg_angles_in_window - angle + np.pi) % (2 * np.pi) - np.pi

        fg_angles_shifted_in_window = relative_theta_fg[
            (relative_theta_fg >= -SCANNING_WINDOW / 2)
            & (relative_theta_fg <= SCANNING_WINDOW / 2)
        ]
        bg_angles_shifted_in_window = relative_theta_bg[
            (relative_theta_bg >= -SCANNING_WINDOW / 2)
            & (relative_theta_bg <= SCANNING_WINDOW / 2)
        ]

        fg_angles_shifted_deg = np.degrees(fg_angles_shifted_in_window)
        bg_angles_shifted_deg = np.degrees(bg_angles_shifted_in_window)

        fg_hist, bg_hist = rsp.compute_histogram(
            fg_angles_shifted_deg, bg_angles_shifted_deg, BIN_EDGES_DEG
        )

        ax2.clear()
        bin_centers_deg = (BIN_EDGES_DEG[:-1] + BIN_EDGES_DEG[1:]) / 2
        width = bin_centers_deg[1] - bin_centers_deg[0]

        ax2.bar(
            bin_centers_deg,
            bg_hist,
            width=width,
            color="gray",
            alpha=0.5,
            label="Background",
        )
        ax2.bar(
            bin_centers_deg,
            fg_hist,
            width=width,
            color="red",
            alpha=0.5,
            label="Foreground",
        )

        ax2.set_xlim(-np.degrees(SCANNING_WINDOW) / 2, np.degrees(SCANNING_WINDOW) / 2)
        ax2.set_xlabel("Angle within Scanning Window (Degrees)")
        ax2.set_ylabel("Count")
        ax2.set_title("Histogram of Angles within Scanning Window")
        ax2.legend(loc="upper right")
        ax2.grid(True, linestyle="--", alpha=0.5)

        fg_cdf, bg_cdf = rsp.compute_cdfs(fg_hist, bg_hist)

        ax3.clear()
        ax3.set_title("CDF of Angles within Scanning Window")
        ax3.set_xlabel("Normalized Angle (0 to 1)")
        ax3.set_ylabel("CDF")
        ax3.set_ylim(0, 1)
        ax3.grid(True, linestyle="--", alpha=0.5)

        # Normalize the angles for plotting CDF
        normalized_bin_edges = (BIN_EDGES_DEG - BIN_EDGES_DEG.min()) / (
            BIN_EDGES_DEG.max() - BIN_EDGES_DEG.min()
        )
        normalized_bin_centers = (
            normalized_bin_edges[:-1] + normalized_bin_edges[1:]
        ) / 2

        ax3.plot(normalized_bin_centers, fg_cdf, label="Foreground CDF", color="red")
        ax3.plot(normalized_bin_centers, bg_cdf, label="Background CDF", color="gray")

        difference = abs(bg_cdf - fg_cdf)
        ax3.fill_between(
            normalized_bin_centers,
            fg_cdf,
            bg_cdf,
            where=(difference > 0),
            color="yellow",
            alpha=0.3,
        )

        ax3.set_xlim(0, 1)
        ax3.legend(loc="upper left")

        diff = rsp.compute_diff(fg_cdf, bg_cdf, mode=MODE)
        rsp_diffs[i] = diff

        ax4.clear()
        ax4.set_title("In-progress RSP Diff Plot")
        angles = theta_k_list[: i + 1]
        ax4.plot(angles, rsp_diffs[: i + 1], color="black")
        ax4.set_rlabel_position(0)
        ax4.legend(loc="upper right")
        ax4.set_ylim(0, 1)
        ax4.grid(True)

        num_fg_in: int = np.sum(fg_in_window)
        num_bg_in: int = np.sum(bg_in_window)
        print(
            f"Frame {i+1}/{RESOLUTION}: {num_fg_in} FG points and {num_bg_in} BG points within the scanning window."
        )
        print(f"RSP Diff: {diff:.4f}")

    ani: FuncAnimation = FuncAnimation(
        fig, animate, frames=RESOLUTION, interval=100, repeat=True
    )
    ani.save("radar_scanning_animation.gif", writer="pillow")

    plt.tight_layout()
    plt.show()

In [ ]:
if __name__ == "__main__":
    rsp.init(
        {
            "MODE": "absolute",
            "SCANNING_WINDOW": np.pi,  # Scanning window in radians
            "RESOLUTION": 100,
        }
    )
    main()